In [6]:
from pathlib import Path
import os
import warnings
import random

import re
import json
import pandas as pd
from datasets import ClassLabel, Dataset, DatasetDict
import librosa
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.display import Audio

from IPython.display import display, HTML


import torch
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor

In [7]:
#!wget https://github.com/besacier/ALFFA_PUBLIC/tree/master/ASR/SWAHILI

In [8]:
!git clone https://github.com/besacier/ALFFA_PUBLIC.git

Cloning into 'ALFFA_PUBLIC'...
remote: Enumerating objects: 66024, done.
remote: Total 66024 (delta 0), reused 0 (delta 0), pack-reused 66024
Receiving objects: 100% (66024/66024), 6.54 GiB | 30.77 MiB/s, done.
Resolving deltas: 100% (12337/12337), done.
Updating files: 100% (56925/56925), done.


In [9]:
with open("/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/text") as file_name:
    data = file_name.read().split("\n")

In [10]:
data[:1]

['SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part10\trais wa tanzania jakaya mrisho kikwete']

In [11]:
file_name=[]
sentences=[]

In [12]:
for i in range(len(data)-1):
    #print(i)
    dta_split = data[i].split("\t")
    file_name.append(dta_split[0])
    sentences.append(dta_split[1])
    

In [13]:
len(data)

10181

In [14]:
file_name[-1]

'SWH-15-20110310_16k-emission_swahili_15h00_-_16h00_tu_20110310_part99'

In [15]:
len(file_name)

10180

In [16]:
sentences[-1]

'na kesi yake ya kubadilishana almasi na silaha na kuchangia mauwaji huko sierra leone'

In [17]:
len(sentences)

10180

In [18]:
audio_data = {
    'ID':file_name,
    'transcription':sentences
}

In [19]:
swahili_train = pd.DataFrame(audio_data)
#swahili_train = wolof_train
swahili_train

,ID,transcription
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya
...,...,...
10175,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,na somo lile lililopokelewa kule kenya
10176,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,ambapo mtu aliyeshindwa kwenye uchaguzi
10177,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,ni kauli yake mchambuzi wa masuala ya siasa
10178,SWH-15-20110310_16k-emission_swahili_15h00_-_1...,mwanasheria anayemtetea rais wa zamani wa liberia


In [20]:
# Create dict of ID's and file paths
paths = {}
path = '/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train'
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".wav"):
            audio_path = os.path.join(root, file)
            p = Path(audio_path)
            id = p.parts[-1].split('.')[0]
            paths[id] = audio_path

In [21]:
swahili_train.iloc[2]

ID               SWH-05-20101106_16k-emission_swahili_05h30_-_0...
transcription    inayokutangazia moja kwa moja kutoka jijini da...
Name: 2, dtype: object

In [22]:
Audio(paths[swahili_train.iloc[2]["ID"].strip()])

In [23]:
swahili_train['file'] = [paths[id.strip()] for id in list(swahili_train['ID'])]

In [24]:
swahili_train = swahili_train.drop(["ID"], axis = 1)

In [25]:
swahili_train

,transcription,file
0,rais wa tanzania jakaya mrisho kikwete,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
1,yanayo andaliwa nami pendo pondo idhaa ya kisw...,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
2,inayokutangazia moja kwa moja kutoka jijini da...,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
3,juma hili bara la afrika limeshuhudia raia wa ...,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
4,wakipiga kura ya maoni ilikufanya mabadiliko ya,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
...,...,...
10175,na somo lile lililopokelewa kule kenya,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
10176,ambapo mtu aliyeshindwa kwenye uchaguzi,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
10177,ni kauli yake mchambuzi wa masuala ya siasa,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...
10178,mwanasheria anayemtetea rais wa zamani wa liberia,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/...


In [26]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [27]:
swahili = DatasetDict({'train': Dataset.from_pandas(swahili_train[0:7000]), 'test': Dataset.from_pandas(swahili_train[7000:])})

In [28]:
chars_to_ignore_regex = '[\"\?\.\!\-\;\:\(\)\,\<\>\-\_\']'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [29]:
swahili = swahili.map(remove_special_characters)

  0%|          | 0/7000 [00:00<?, ?ex/s]

  0%|          | 0/3180 [00:00<?, ?ex/s]

In [30]:
show_random_elements(swahili['train'], num_examples=10)

,transcription,file
0,inayoongozwa na siaka siangale kushindwa kutoa matokeo,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20101115/SWH-15-20101115_16k-emission_swahili_15h00_-_16h00_tu_20101115_part294.wav
1,kusema kuibadilisha mbona sehemu zingine nawezekana,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20101104/SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part493.wav
2,lakini kuna watu kadhaa ambao hawa,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20101115/SWH-15-20101115_16k-emission_swahili_15h00_-_16h00_tu_20101115_part399.wav
3,aa jumapili ya leo asante sana na,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101121/SWH-05-20101121_16k-emission_swahili_05h30_-_06h00_tu_20101121_part187.wav
4,wapo ambao mpaka sasa hivi hawajulikani walipo,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20110114/SWH-05-20110114_16k-emission_swahili_05h30_-_06h00_tu_20110114_part19.wav
5,naam ndivyo anavyoondoka pendo po na makala,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101113/SWH-05-20101113_16k-emission_swahili_05h30_-_06h00_tu_20101113_part197.wav
6,obama amesema atatumia vizuri nafasi yake atakayoipata siku hiyo ya jumanne,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20110123/SWH-05-20110123_16k-emission_swahili_05h30_-_06h00_tu_20110123_part10.wav
7,sehemu za tanzania na wenyewe wanavutia kwao rwan,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20101109/SWH-15-20101109_emission_swahili_15h00_-_16h00_tu_20101109_part223.wav
8,aa tunaangazia hali ya nidhamu miongoni mwa wachezaji wa timu ya taifa soka afrika mashariki na kati,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20110327/SWH-05-20110327_16k-emission_swahili_05h30_-_06h00_tu_20110327_part89.wav
9,litafanyika katika muda mfupi ujao na kwamba baadhi ya mawaziri,/kaggle/working/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-15-20101104/SWH-15-20101104_emission_swahili_15h00_-_16h00_tu_20101104_part341.wav


In [31]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [32]:
vocabs = swahili.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=swahili.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
vocab_list = list(set(vocabs["train"]["vocab"][0]))


In [34]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'3': 0,
 'a': 1,
 '6': 2,
 't': 3,
 'm': 4,
 '7': 5,
 'y': 6,
 'k': 7,
 'z': 8,
 'u': 9,
 'v': 10,
 'g': 11,
 'd': 12,
 ' ': 13,
 'o': 14,
 'p': 15,
 '2': 16,
 '9': 17,
 'x': 18,
 '0': 19,
 'l': 20,
 'f': 21,
 's': 22,
 '1': 23,
 'w': 24,
 'q': 25,
 'n': 26,
 'b': 27,
 'c': 28,
 'h': 29,
 '5': 30,
 'j': 31,
 'i': 32,
 'e': 33,
 'r': 34}

In [35]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [36]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

37

In [37]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [38]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [39]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [40]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [41]:
#import soundfile as sf
import librosa
import warnings

warnings.filterwarnings("ignore")

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["file"], sr = 16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription"]
    return batch

In [42]:
swahili = swahili.map(speech_file_to_array_fn, remove_columns=swahili.column_names["train"], num_proc=1)

  0%|          | 0/7000 [00:00<?, ?ex/s]

  0%|          | 0/3180 [00:00<?, ?ex/s]

In [43]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [44]:
swahili_prepared = swahili.map(prepare_dataset, remove_columns=swahili.column_names["train"], batch_size=32, batched=True)

  0%|          | 0/219 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

In [45]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [46]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [47]:
!pip install  jiwer

In [48]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [49]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [50]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
model.freeze_feature_extractor()

In [52]:
torch.cuda.is_available() 

True

In [53]:
os.mkdir("/kaggle/working/swahili")

In [58]:
from huggingface_hub import notebook_login


notebook_login()


In [59]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="Peed911/swahili_asr_audio_to_text",
  group_by_length=True,
  per_device_train_batch_size=3,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  save_steps=40000,
  eval_steps=1000,
  logging_steps=1000,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=12,
  push_to_hub = True
)

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/kaggle/working/swahili",
  group_by_length=True,
  per_device_train_batch_size=3,
  evaluation_strategy="steps",
  num_train_epochs=12,
  fp16=True,
  save_steps=40000,
  eval_steps=1000,
  logging_steps=1000,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=12,
)

In [60]:
from transformers import Trainer


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=swahili_prepared["train"],
    eval_dataset=swahili_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [61]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
1000,0.948700,0.781080,0.745764
2000,0.643300,0.615809,0.631885


TrainOutput(global_step=2334, training_loss=0.7525959031236611, metrics={'train_runtime': 2954.1553, 'train_samples_per_second': 4.739, 'train_steps_per_second': 0.79, 'total_flos': 1.5068901415559516e+18, 'train_loss': 0.7525959031236611, 'epoch': 2.0})

In [64]:
model.push_to_hub("Peed911/swahili_asr_audio_to_text")

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Peed911/swahili_asr_audio_to_text/commit/0ca419446df994eae9a5c9df2f85e61d3b175125', commit_message='Upload Wav2Vec2ForCTC', commit_description='', oid='0ca419446df994eae9a5c9df2f85e61d3b175125', pr_url=None, pr_revision=None, pr_num=None)

In [65]:
tokenizer.push_to_hub("Peed911/swahili_asr_audio_to_text")

CommitInfo(commit_url='https://huggingface.co/Peed911/swahili_asr_audio_to_text/commit/0096be01bb82796a211340623e6a6dc9f3a9e9e3', commit_message='Upload tokenizer', commit_description='', oid='0096be01bb82796a211340623e6a6dc9f3a9e9e3', pr_url=None, pr_revision=None, pr_num=None)